# Datasets and profile preparation using Slab2.0

In [1]:
import os
import sys
import openquake.sub as sub

# Get the path of the mbtk subduction module
path = sub.__path__[0]

print(path)

/Users/mpagani/Repos/venv/src/oq-mbtk/openquake/sub


## 1. Pickle the catalogue
First we pickle the catalogue, that is, we create a compact representation of this dataset that can be read quickly by python.

In [2]:
!python $path/pickle_catalogue.py ./../data/catalogue/wpg_catalogue_sumatra.csv
!mv catalogue_ext.p ./../data/catalogue/wpg_catalogue_sumatra.p
!rm catalogue_ori.p

## 2. Create cross-sections

In [3]:
!python $path/create_multiple_cross_sections.py ./../ini/create_profiles.txt
!mkdir ./../model
!mv cs_traces.cs ./../model/cs_traces.txt

104.248585 -8.110244 400.000000 400.000000 16.808520 0 ./../ini/create_profiles.txt
103.421644 -7.772352 400.000000 400.000000 39.982619 1 ./../ini/create_profiles.txt
102.727092 -7.193938 400.000000 400.000000 39.982619 2 ./../ini/create_profiles.txt
102.069200 -6.576154 400.000000 400.000000 44.060749 3 ./../ini/create_profiles.txt
101.485762 -5.895142 400.000000 400.000000 56.062353 4 ./../ini/create_profiles.txt
100.981659 -5.148806 400.000000 400.000000 56.062353 5 ./../ini/create_profiles.txt
100.435749 -4.433603 400.000000 400.000000 51.189388 6 ./../ini/create_profiles.txt
99.871355 -3.732278 400.000000 400.000000 52.046981 7 ./../ini/create_profiles.txt
99.317502 -3.022986 400.000000 400.000000 52.046981 8 ./../ini/create_profiles.txt
98.763969 -2.313727 400.000000 400.000000 52.046981 9 ./../ini/create_profiles.txt
98.257054 -1.571587 400.000000 400.000000 57.256067 10 ./../ini/create_profiles.txt
97.770589 -0.815124 400.000000 400.000000 57.256067 11 ./../ini/create_profiles

## 3. Calculate profiles

In [4]:
fname_sl = './../data/subduction/sum_slab2_dep_02.23.18.xyz'
fout = './../model/profiles/'
fname_cs = './../model/cs_traces.txt'

In [5]:
!oqm sub create_sections_from_slab $fname_sl $fout $fname_cs

## 4. Create the profiles and edges definining the top of the inslab 

In [9]:
sampling=10.0
uppdep=40
lowdep=300
in_folder='./../model/profiles/'
out_folder='./../model/surfaces/sum'

In [10]:
!oqm sub build_complex_surface $in_folder $sampling $out_folder $uppdep $lowdep

sub build_complex_surface ./../model/profiles/ 10.0 ./../model/surfaces/sum 40 300
INFO:root:Number of profiles: 21
INFO:root:Longest profile (id: 006): 361.680132
INFO:root:Shortest profile (id: 001): 258.342717
INFO:root:Depth min: 40.00
INFO:root:Depth max: 300.00
INFO:root:Number of subsegments for each profile: 37
INFO:root:Shortest sampling [001]: 6.9822
INFO:root:Longest sampling  [006]: 9.7751


## 5. Plotting

Now we create a .geojson file containing the profiles and edges which can be visualized with a GIS.

In [11]:
pattern = './../model/profiles/cs*.csv'
output = './../output/geojson/profiles.geojson'

In [13]:
!oqm sub geojson_from_profiles "$pattern" $output

./../model/profiles/cs*.csv
./../model/profiles/cs_006.csv
./../model/profiles/cs_012.csv
./../model/profiles/cs_013.csv
./../model/profiles/cs_007.csv
./../model/profiles/cs_011.csv
./../model/profiles/cs_005.csv
./../model/profiles/cs_004.csv
./../model/profiles/cs_010.csv
./../model/profiles/cs_014.csv
./../model/profiles/cs_000.csv
./../model/profiles/cs_001.csv
./../model/profiles/cs_015.csv
./../model/profiles/cs_003.csv
./../model/profiles/cs_017.csv
./../model/profiles/cs_016.csv
./../model/profiles/cs_002.csv
./../model/profiles/cs_018.csv
./../model/profiles/cs_019.csv
./../model/profiles/cs_009.csv
./../model/profiles/cs_008.csv
./../model/profiles/cs_020.csv


In [14]:
import json
from ipyleaflet import Map, GeoJSON

In [15]:
with open(output, 'r') as f:
    data = json.load(f)
geo_json = GeoJSON(data=data, style={'opacity': 1, 'dashArray': '2', 'fillOpacity': 0.1, 'weight': 1})
m = Map(center=(0.0, 100.0), zoom=5)
m.add_layer(geo_json)
m

Map(center=[0.0, 100.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…